<a href="https://colab.research.google.com/github/svetlanama/math/blob/main/%D0%94%D0%9710_%D0%9C%D0%BE%D1%96%D1%81%D0%B5%D1%94%D0%BD%D0%BA%D0%BE_%D0%A1%D0%B2%D1%96%D1%82%D0%BB%D0%B0%D0%BD%D0%B0_%D0%9E%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%D1%96%D0%B2%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



# Шлях до файлу (залежить куди ти його поклав)
python_material_folder_name = "neo_data/House_Rent_Dataset.csv" #"Rent Data.csv"

# Check if in Google Colab environment
try:
    from google.colab import drive
    # Mount drive
    drive.mount('/content/drive')
    # Set up path to Python material parent folder
    path_python_material = rf"drive/MyDrive/{python_material_folder_name}"
        # If unsure, print current directory path by executing the following in a new cell:
        # !pwd
    IN_COLAB = True

    import os
    import sys

    # Get the current working directory (notebook's directory)
    notebook_dir = f"{path_python_material}/notebooks"
    print(notebook_dir)

    if notebook_dir not in sys.path:
      sys.path.append(notebook_dir)

except:
    IN_COLAB = False
    # If working locally on Jupyter Notebook, parent folder is one folder up (assuming you are using the folder structure shared at the beginning of the course)
    path_python_material = ".."

print("path_python_material:", path_python_material)


df = pd.read_csv(path_python_material)
display(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/MyDrive/neo_data/House_Rent_Dataset.csv/notebooks
path_python_material: drive/MyDrive/neo_data/House_Rent_Dataset.csv


,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("path_python_material:", path_python_material)
# --- 1. Завантаження датасету ---
# file_path = "/content/drive/MyDrive/neo_data/House_Rent_Dataset.csv"
path_python_material = "/content/drive/MyDrive/neo_data/House_Rent_Dataset.csv"
df = pd.read_csv(path_python_material)
# print("✅ Форма датасету:", df.shape)
display(df.head())

# --- 2. Базова статистика ---
print("\n=== INFO ===")
print(df.info())




path_python_material: drive/MyDrive/neo_data/House_Rent_Dataset.csv


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/neo_data/House_Rent_Dataset.csv'

In [ ]:
print("\n=== Описова статистика (числові колонки) ===")
display(df.describe().T)  # середні, std, min, max (діапазони видно)

# дисперсія для числових колонок
print("\n=== Дисперсія числових ознак ===")
display(df.var(numeric_only=True))

# --- 3. Перевірка пропусків ---
na_tbl = (
    pd.DataFrame({
        "na_count": df.isna().sum(),
        "na_%": df.isna().mean()*100
    })
    .sort_values("na_count", ascending=False)
)
print("\n=== Пропущені значення ===")
display(na_tbl[na_tbl.na_count > 0])

# Заповнення: числові -> медіана, категоріальні -> мода
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].median())
    else:
        if df[col].isna().any():
            mode = df[col].mode(dropna=True)
            if len(mode) > 0:
                df[col] = df[col].fillna(mode.iloc[0])

print("✅ Пропуски заповнено")

# --- 4. Типи ознак ---
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols  = df.select_dtypes(include=["object"]).columns.tolist()

print("\nЧислові:", numeric_cols)
print("Категоріальні:", cat_cols)

# --- 5. Бінарні ознаки ---
binary_like_cols = []
for col in cat_cols:
    uniq = df[col].dropna().unique()
    if len(uniq) == 2:  # якщо рівно 2 значення
        binary_like_cols.append(col)

def make_binary_01(series: pd.Series):
    vals = sorted(series.dropna().unique(), key=lambda v: str(v))
    return series.map({vals[0]: 0, vals[1]: 1}).astype(int)

for col in binary_like_cols:
    df[col] = make_binary_01(df[col])

print("✅ Бінарні object-колонки перетворені у 0/1:", binary_like_cols)

# --- 6. Візуалізація розподілів ---
if numeric_cols:
    df[numeric_cols].hist(figsize=(14, 10), bins=30)
    plt.suptitle("Розподіли числових ознак", y=1.02)
    plt.show()

# Boxplot для виявлення викидів
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[numeric_cols])
plt.title("Boxplot числових ознак")
plt.xticks(rotation=45)
plt.show()

# --- 7. Кореляції ---
if len(numeric_cols) > 1:
    plt.figure(figsize=(10, 8))
    corr = df[numeric_cols].corr(numeric_only=True)
    sns.heatmap(corr, annot=False, cmap="coolwarm", square=True)
    plt.title("Кореляції числових ознак")
    plt.show()

print("\n✅ Аналіз та базове чищення завершено")